In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
spark = SparkSession.builder.getOrCreate()
import datetime

spark\
.read\
.option('header',True)\
.csv('/mnt/c/Users/ProSimple/Downloads/CAR DETAILS FROM CAR DEKHO.csv')\
.limit(100)\
.createOrReplaceTempView('cars_dataset')

df = spark.table('cars_dataset')

df.toPandas()


22/03/25 09:13:32 WARN Utils: Your hostname, LAPTOP-V8EALT7T resolves to a loopback address: 127.0.1.1; using 172.20.10.2 instead (on interface wifi0)
22/03/25 09:13:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/25 09:13:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner
...,...,...,...,...,...,...,...,...
95,Fiat Avventura MULTIJET Emotion,2015,350000,53000,Diesel,Individual,Manual,Second Owner
96,Audi A8 4.2 TDI,2013,2800000,49000,Diesel,Dealer,Automatic,First Owner
97,Datsun RediGO 1.0 S,2017,210000,15000,Petrol,Dealer,Manual,Second Owner
98,Volkswagen Jetta 1.4 TSI Comfortline,2013,450000,50000,Petrol,Individual,Manual,First Owner


In [6]:
def crazy_staff(model):
    spl_model = model.split()[1::]
    spl_join = ' '.join(spl_model)
    return spl_join

# crazy_staff('Maruti 800 AC') --> [Maruti] [800] [AC] --> [800 AC]

crazy_staff_udf = F.udf(crazy_staff)

In [7]:
split_col = F.split(df['name'], ' ')

df2 = df.withColumn('Brand', split_col[0])\
        .withColumn('Model',F.array_join(F.expr('array_remove(split(name," "), split(name," ")[0])'),' '))\
        .withColumn('Price_Usd', F.expr('round(selling_price / 76.43,2)'))\
        .withColumn('Price_Eur', F.expr('round(selling_price / 83.45,2)'))\
        .withColumn('Price_Gbp', F.expr('round(selling_price / 101.09,2)'))\
        .withColumn('Current_Date', F.current_date()).withColumn('Current_Year', F.year('Current_Date'))\
     	.withColumn('Using_Years', (F.col('Current_Year') - F.col('year')).cast(IntegerType()))\
        .withColumnRenamed('km_driven','Km_Driven')\
        .withColumnRenamed('fuel','Fuel')\
        .withColumnRenamed('transmission','Transmission')\
        .withColumnRenamed('owner','Owner')\
        .withColumnRenamed('year','Year')

In [8]:
df3 = df2\
.withColumn('Country_Brand',
            F.when(df2.Brand == "Audi", 'Germany')\
            .when(df2.Brand == "BMW", 'Germany')\
            .when(df2.Brand == 'Mercedes-Benz', 'Germany')\
            .when(df2.Brand == 'Jaguar', 'England')\
            .when(df2.Brand == 'Toyota', 'Japan')\
            .when(df2.Brand == 'Volkswagen', 'Germany')\
            .otherwise('Other'))


In [10]:
df3\
.where('Brand in ("Audi", "BMW", "Jaguar", "Mercedes-Benz", "Toyota", "Volkswagen", "Ford")')\
.groupBy('Brand','Model','Country_Brand','Transmission')\
.agg(
     F.count('Model').alias('Cnt_Cars'),
     F.round(F.avg('Price_Usd'),2).alias('Avg_Price_Usd'),
     F.round(F.avg('Price_Eur'),2).alias('Avg_Price_Eur'))\
.orderBy('Brand','Transmission')\
.limit(10).toPandas()

,Brand,Model,Country_Brand,Transmission,Cnt_Cars,Avg_Price_Usd,Avg_Price_Eur
0,Audi,Q5 2.0 TDI,Germany,Automatic,1,15569.80,14260.04
1,Audi,A4 2.0 TDI 177 Bhp Premium Plus,Germany,Automatic,1,15046.45,13780.71
2,Audi,Q7 35 TDI Quattro Premium,Germany,Automatic,1,16354.83,14979.03
3,Audi,A6 2.0 TDI Premium Plus,Germany,Automatic,1,19233.29,17615.34
4,Audi,A6 2.0 TDI Design Edition,Germany,Automatic,1,22700.51,20790.89
5,Audi,A8 4.2 TDI,Germany,Automatic,1,36634.83,33553.03
6,BMW,3 Series 320d Sport Line,Germany,Automatic,1,20279.99,18574.00
7,Ford,EcoSport 1.5 Diesel Titanium BSIV,Other,Manual,1,12102.58,11084.48
8,Jaguar,XF 2.2 Litre Luxury,England,Automatic,1,25709.79,23547.02
9,Mercedes-Benz,New C-Class 220 CDI AT,Germany,Automatic,1,18644.51,17076.09
